In [21]:
import torch
from torch import nn 
from torch import optim
from river import compose, metrics, preprocessing, stream, anomaly, linear_model, datasets, compose
from river import feature_extraction as fx


from IncrementalTorch.anomaly.anomaly import Autoencoder, BasicAutoencoder
from river import compat

from tqdm import tqdm
import river  
import torchvision
import numpy as np
import pandas as pd
from pprint import pprint


from OnlineTorch.classifier import PyTorch2RiverClassifier
from OnlineTorch.anomaly import TorchAE

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDOneClassSVM
from sklearn.cluster import k_means
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import confusion_matrix as cm
from sklearn.datasets import fetch_covtype
from sklearn.svm import SVC


N_SAMPLES = 1_000 #number of elements stored in memory
SEED = 42
#track_name = "RBF"
#LOSS = nn.BCELoss
LOSS = nn.L1Loss
#LOSS = nn.CrossEntropyLoss
OPTIMIZER = optim.AdamW
BATCH_SIZE=10
LEARNING_RATE=1e-3
LATENT_DIM = 1
rocauc = river.metrics.ROCAUC()
rocauc=0

threshhold=0

##max 5% with 10000 samples
anom_percentage = 10
num_samples = 5000

anom_percentage1 = 50
num_samples1 = 25000
num_anom_classes=1 #if you want 1 anom class type 1


#F1,F2,F0,5 interessant?


In [22]:
torch.__version__

'1.10.2+cpu'

In [23]:
evaluation=pd.DataFrame(columns=['model','anom_percentage','num_samples','Loss','Optimizer','Batch_size','Learning_rate','Latent_dim','ROC','Precision_Recall_Curve','FP','TP','FN','TN'])


In [24]:
'''
class 5 contains 9493 instances
class 2 contains 283301 instances
class 1 contains 211840 instances
class 7 contains 20510 instances
class 3 contains 35754 instances
class 6 contains 17367 instances
class 4 contains 2747 instances
'''

'\nclass 5 contains 9493 instances\nclass 2 contains 283301 instances\nclass 1 contains 211840 instances\nclass 7 contains 20510 instances\nclass 3 contains 35754 instances\nclass 6 contains 17367 instances\nclass 4 contains 2747 instances\n'

In [25]:
'''evaluation.append({
    'model':1,
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)
'''

"evaluation.append({\n    'model':1,\n    'anom_percentage':anom_percentage,\n    'num_samples':num_samples,\n    'Loss':LOSS,\n    'Optimizer':OPTIMIZER,\n    'Batch_size':BATCH_SIZE,\n    'Learning_rate':LEARNING_RATE,\n    'Latent_dim': LATENT_DIM,\n    'ROC':rocauc,\n    #'Precision_Recall_Curve':45,\n    #'FP':5,\n    #'TP': 4,\n    #'FN':8,\n    #'TN':10\n\n},ignore_index=True)\n"

# Dataset

## Covtype Dataset

In [26]:
covtype_x, covtype_y=fetch_covtype(as_frame=True,return_X_y=True)
covtype_df=covtype_x
covtype_df['target']=covtype_y
covtype_df['target']=covtype_df['target']-1

In [27]:
#covtype_df[covtype_df['target']==0].count()

In [28]:
covtype_df_classes=covtype_df.target.unique()
for i in covtype_df_classes:
    print('class {0} contains {1} instances'.format(i,covtype_df[covtype_df.target==i].Elevation.count()))

class 4 contains 9493 instances
class 1 contains 283301 instances
class 0 contains 211840 instances
class 6 contains 20510 instances
class 2 contains 35754 instances
class 5 contains 17367 instances
class 3 contains 2747 instances


In [29]:
for i in range (1,num_anom_classes+1):
    covtype_df.target[covtype_df.target==i]=1
    covtype_df.loc[covtype_df['target']==i, 'target']=1
covtype_df_filtered=covtype_df[covtype_df.target<=num_anom_classes]
covtype_df_filtered.dropna(inplace=True)
covtype_df_filtered.reset_index(inplace=True,drop=True)

C:\Users\Manuel\AppData\Local\Temp/ipykernel_16200/2152947708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covtype_df.target[covtype_df.target==i]=1
C:\Users\Manuel\AppData\Roaming\Python\Python38\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [30]:
covtype_df_classes=covtype_df.target.unique()
for i in covtype_df_classes:
    print('class {0} contains {1} instances'.format(i,covtype_df[covtype_df.target==i].Elevation.count()))

class 4 contains 9493 instances
class 1 contains 283301 instances
class 0 contains 211840 instances
class 6 contains 20510 instances
class 2 contains 35754 instances
class 5 contains 17367 instances
class 3 contains 2747 instances


In [31]:
num_anom = int(num_samples1*(anom_percentage1/100))
num_clean = int(num_samples1-num_anom)

anoms=covtype_df_filtered[covtype_df_filtered['target']==1].iloc[0:num_anom]
clean = covtype_df_filtered[covtype_df_filtered['target']==0].iloc[0:num_clean]
frames = [anoms,clean]
print('Stream contains {} anomalies and {} no-anomalies'.format(anoms['target'].count(),clean['target'].count()))

final_set = pd.concat(frames,ignore_index=True)


data_stream1 = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)

Stream contains 12500 anomalies and 12500 no-anomalies


In [32]:
x_covtype=final_set.iloc[:,:-1].transpose().to_dict()
y_covtype=final_set.iloc[:,-1].transpose().to_dict()

In [33]:
df_x_covtype= pd.DataFrame.from_dict(x_covtype)
df_y_covtype= pd.Series(y_covtype)

In [34]:
frames_test_test=pd.DataFrame(data=[x_covtype,y_covtype]).transpose()
final_set=frames_test_test.copy()

In [35]:
final_set_x=final_set.iloc[:,:-1]
final_set_y=final_set.iloc[:,-1:]

In [36]:
#covtype_df_filtered=covtype_df[covtype_df.target!=2].reset_index(inplace=True)

In [37]:
#covtype_df_filtered

## Credit Card Dataset

In [38]:
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)

In [39]:
#data_stream = stream.shuffle(river.datasets.Insects().take(8000), N_SAMPLES, seed=SEED)
#for x, y in data_stream:
#    print(x, y)

In [40]:

num_anom = int(num_samples*(anom_percentage/100))
num_clean = int(num_samples-num_anom)

data_stream = stream.shuffle(river.datasets.CreditCard(), N_SAMPLES, seed=SEED)
data1 = pd.DataFrame(data=data_stream)
df_test= pd.DataFrame.from_dict(data1)

anoms=df_test[df_test[1]==1].iloc[0:num_anom]
clean = df_test[df_test[1]==0].iloc[0:num_clean]
frames = [anoms,clean]
print('Stream contains {} anomalies and {} no-anomalies'.format(anoms[0].count(),clean[0].count()))
final_set = pd.concat(frames,ignore_index=True)


data_stream1 = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)

Stream contains 492 anomalies and 4500 no-anomalies


In [41]:
#data1

In [42]:
evaluation

,model,anom_percentage,num_samples,Loss,Optimizer,Batch_size,Learning_rate,Latent_dim,ROC,Precision_Recall_Curve,FP,TP,FN,TN


# Autoencoder

## OLD Undercomplete Autoencoder

### Softmax

In [43]:
'''def undercomplete_ae_sm(n_features, latent_dim=1):
    net = nn.Sequential(
        nn.Dropout(),
        nn.Linear(n_features, 20), 
        nn.LeakyReLU(),
        nn.Linear(20, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, 20),
        nn.LeakyReLU(), 
        nn.Linear(20, n_features),
        nn.Linear(n_features,1),
        nn.Softmax()
    )
    return net
    '''

'def undercomplete_ae_sm(n_features, latent_dim=1):\n    net = nn.Sequential(\n        nn.Dropout(),\n        nn.Linear(n_features, 20), \n        nn.LeakyReLU(),\n        nn.Linear(20, latent_dim),\n        nn.LeakyReLU(),\n        nn.Linear(latent_dim, 20),\n        nn.LeakyReLU(), \n        nn.Linear(20, n_features),\n        nn.Linear(n_features,1),\n        nn.Softmax()\n    )\n    return net\n    '

In [44]:
'''undercomplete_ae_sm_model = compose.Pipeline(
    preprocessing.StandardScaler(),
    PyTorch2RiverClassifier(
                build_fn = undercomplete_ae_sm,
                loss_fn = LOSS,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )
)
'''

'undercomplete_ae_sm_model = compose.Pipeline(\n    preprocessing.StandardScaler(),\n    PyTorch2RiverClassifier(\n                build_fn = undercomplete_ae_sm,\n                loss_fn = LOSS,\n                optimizer_fn = OPTIMIZER,\n                #batch_size=BATCH_SIZE,\n                learning_rate=LEARNING_RATE,\n                seed=SEED\n    )\n)\n'

In [45]:
'''#supervised learning approach with Softmax function --> Proba gets predicted, ROC way worse
rocauc = river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    y_pred = undercomplete_ae_sm_model.predict_proba_one(x) #ruft learn_unsupervised auf, müssen wir learn_one dann überhaupt auch aufrufen?
    undercomplete_ae_sm_model.learn_one(x, y) #undercomplete_ae_sm_model.learn_one ist supervised Ansatz, wir wollen Unsupervised
    rocauc.update(y, y_pred)
rocauc
'''

'#supervised learning approach with Softmax function --> Proba gets predicted, ROC way worse\nrocauc = river.metrics.ROCAUC()\n#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)\ndata_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)\nfor x, y in data_stream:\n    y_pred = undercomplete_ae_sm_model.predict_proba_one(x) #ruft learn_unsupervised auf, müssen wir learn_one dann überhaupt auch aufrufen?\n    undercomplete_ae_sm_model.learn_one(x, y) #undercomplete_ae_sm_model.learn_one ist supervised Ansatz, wir wollen Unsupervised\n    rocauc.update(y, y_pred)\nrocauc\n'

## Undercomplete Autoencoder standard

In [46]:
def undercomplete_ae(n_features, latent_dim=1):
    net = nn.Sequential(
        nn.Dropout(),
        nn.Linear(n_features, 20),
        nn.LeakyReLU(),
        nn.Linear(20, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, 20),
        nn.LeakyReLU(), 
        nn.Linear(20, n_features),
#        nn.Linear(n_features, 1),
#        nn.Sigmoid()
    )
    return net

    #Sigmoid bringt nicht so viel, da immer angepasst auf das aktuelle Trainingsbeispiel --> ist nicht zwischen 0,1

In [47]:
undercomplete_ae_model = compose.Pipeline(
    #preprocessing.StandardScaler(),
    compat.PyTorch2RiverClassifier(
                build_fn = undercomplete_ae,
                loss_fn = nn.BCELoss,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )
)

In [48]:
undercomplete_ae_model = Autoencoder(
                build_fn = undercomplete_ae,
                loss_fn = LOSS,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )


In [49]:
undercomplete_ae_model = compose.Pipeline(
    preprocessing.StandardScaler(),
    TorchAE(
                build_fn = undercomplete_ae,
                loss_fn = LOSS,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )
)

In [50]:
'''undercomplete_ae_model = compose.Pipeline(
    preprocessing.StandardScaler(),
    PyTorch2RiverClassifier(
                build_fn = undercomplete_ae,
                loss_fn = nn.MSELoss,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )
)
'''

'undercomplete_ae_model = compose.Pipeline(\n    preprocessing.StandardScaler(),\n    PyTorch2RiverClassifier(\n                build_fn = undercomplete_ae,\n                loss_fn = nn.MSELoss,\n                optimizer_fn = OPTIMIZER,\n                #batch_size=BATCH_SIZE,\n                learning_rate=LEARNING_RATE,\n                seed=SEED\n    )\n)\n'

In [51]:
rocauc= river.metrics.ROCAUC(n_thresholds=100)
y_pred_arr_undercomplete_ae = []
y_act_arr_undercomplete_ae = []
y_pred_cf=[]
#f1 = river.metrics.F1()
#cm=river.metrics.ConfusionMatrix()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    undercomplete_ae_model.learn_one(x)
    y_pred= undercomplete_ae_model.score_one(x)
#    print(y_pred)
    rocauc.update(y,y_pred)
#    if y_pred>threshhold:
#        y_pred=1
#    else:
#        y_pred=0
    y_pred_arr_undercomplete_ae.append([y_pred,y])
    y_pred_cf.append(y_pred)
    y_act_arr_undercomplete_ae.append(y)

    #f1.update(y,y_pred)
    #cm.update(y,y_pred)
    #undercomplete_ae_model.learn_one(x,y_pred)
    #undercomplete_ae_model.learn_one(x,learn_unsupervised=True)
    #y_pred = undercomplete_ae_sm_model.predict_proba_one(x) #ruft learn_unsupervised auf, müssen wir learn_one dann überhaupt auch aufrufen?
    #undercomplete_ae_model.learn_one(x, y)
    #rocauc.update(y, y_pred)
rocauc
evaluation=evaluation.append({
    'model':'undercomplete_ae_model',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)
#f1

In [52]:
#metric1 = metrics.F1()
#metric1.revert(y_act_arr_undercomplete_ae,y_pred_cf)

#metric1


In [53]:
#y_pred_arr_undercomplete_ae

In [54]:
rocauc

ROCAUC: 0.908965

In [55]:
rocauc= river.metrics.ROCAUC(n_thresholds=100)
y_pred_arr_undercomplete_ae = []
y_act_arr_undercomplete_ae = []
y_pred_cf=[]
#f1 = river.metrics.F1()
#cm=river.metrics.ConfusionMatrix()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    undercomplete_ae_model.learn_one(x)
    y_pred= undercomplete_ae_model.score_one(x) #high score means outlier
#    print(y_pred)
    rocauc.update(y,y_pred)
#    if y_pred>threshhold:
#        y_pred=1
#    else:
#        y_pred=0
    y_pred_arr_undercomplete_ae.append([y_pred,y])
    y_pred_cf.append(y_pred)
    y_act_arr_undercomplete_ae.append(y)

    #f1.update(y,y_pred)
    #cm.update(y,y_pred)
    #undercomplete_ae_model.learn_one(x,y_pred)
    #undercomplete_ae_model.learn_one(x,learn_unsupervised=True)
    #y_pred = undercomplete_ae_sm_model.predict_proba_one(x) #ruft learn_unsupervised auf, müssen wir learn_one dann überhaupt auch aufrufen?
    #undercomplete_ae_model.learn_one(x, y)
    #rocauc.update(y, y_pred)
rocauc
evaluation=evaluation.append({
    'model':'undercomplete_ae_model',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)
#f1

In [56]:
rocauc

ROCAUC: 0.929231

In [57]:
#y_pred_arr_undercomplete_ae

In [58]:
df_tes=pd.DataFrame(y_pred_arr_undercomplete_ae,columns=['y_pred','y_true'])

In [59]:
#df_tes

In [60]:
#df_tes[df_tes['y_true']==1].head()

In [61]:
df_tes=df_tes.sort_values(by=['y_pred'],ascending=False,ignore_index=True)

In [62]:
#df_tes.head(50)

In [63]:
#df_tes[df_tes['y_true']==1].sort_values(by='y_pred',ascending=True).head(100)

In [64]:
#evaluation

## Stacked Autoencoder

In [65]:
def stacked_ae(n_features, latent_dim=1):
    net = nn.Sequential(
        nn.Dropout(),
        nn.Linear(n_features, 20),
        nn.LeakyReLU(),
        nn.Linear(20, 15),
        nn.LeakyReLU(),        
        nn.Linear(15, 10),
        nn.LeakyReLU(),        
        nn.Linear(10, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, 10),
        nn.LeakyReLU(),
        nn.Linear(10, 15),
        nn.LeakyReLU(),
        nn.Linear(15, 20),
        nn.LeakyReLU(),               
        nn.Linear(20, n_features),
    )
    return net

In [66]:
stacked_ae_model = compose.Pipeline(
    preprocessing.MinMaxScaler(),
    TorchAE(
                build_fn = stacked_ae,
                loss_fn = LOSS,
                optimizer_fn = OPTIMIZER,
                #batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                seed=SEED
    )
)

In [67]:
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    stacked_ae_model.learn_one(x)
    y_pred= stacked_ae_model.score_one(x)
    rocauc.update(y,y_pred)
rocauc
evaluation=evaluation.append({
    'model':'undercomplete_stacked_ae_model',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)

In [68]:
#evaluation

# Non-AE-(Incremental Baselines)

## Unsupervised

### OneClassSVM

In [69]:
river.__version__  #0.9.0 required for OneClassSVM

'0.9.0'

In [70]:
''#funktioniert nur mit river=0.9, allerdings muss dann git repo geupdatet werden, da anomalie.anomaliedetector klasse nicht mehr in base sondern in anomly ist --> Wheels können nicht mehr so gebaut werden wie bisher
model4 = compose.Pipeline(
    preprocessing.StandardScaler(),
    #fx.RBFSampler(),    
    anomaly.QuantileThresholder(
        anomaly.OneClassSVM(),
        q=0.97 #q Anpassung viele Auswirkungen
    )
)
''

''

In [71]:
rocauc= river.metrics.ROCAUC()
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    model4.learn_one(x)
    y_pred= model4.score_one(x)
    rocauc.update(y,y_pred)
rocauc


ROCAUC: 0.543084

### HalfSpaceTrees

In [72]:
model3 = compose.Pipeline(
    preprocessing.MinMaxScaler(),
    anomaly.HalfSpaceTrees(seed=SEED)
)

In [73]:
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    model3.learn_one(x)
    y_pred= model3.score_one(x)
    #print(y_pred)
    rocauc.update(y,y_pred)
#evaluation.append(rocauc)
rocauc
evaluation=evaluation.append({
    'model':'HalfSpaceTrees_min_max_scaler',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)

In [74]:
rocauc

ROCAUC: 0.732409

In [75]:
evaluation

,model,anom_percentage,num_samples,Loss,Optimizer,Batch_size,Learning_rate,Latent_dim,ROC,Precision_Recall_Curve,FP,TP,FN,TN
0,undercomplete_ae_model,10,5000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,10,0.001,1,ROCAUC: 0.908965,NaN,NaN,NaN,NaN,NaN
1,undercomplete_ae_model,10,5000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,10,0.001,1,ROCAUC: 0.929231,NaN,NaN,NaN,NaN,NaN
2,undercomplete_stacked_ae_model,10,5000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,10,0.001,1,ROCAUC: -0.,NaN,NaN,NaN,NaN,NaN
3,HalfSpaceTrees_min_max_scaler,10,5000,<class 'torch.nn.modules.loss.L1Loss'>,<class 'torch.optim.adamw.AdamW'>,10,0.001,1,ROCAUC: 0.732409,NaN,NaN,NaN,NaN,NaN


In [76]:
evaluation.to_csv('evaluation_1.vers')

### PCA (Incremental)

In [77]:
ipca = IncrementalPCA(n_components=LATENT_DIM)
model6=compose.Pipeline(
    preprocessing.MinMaxScaler(),
    ipca #IncrementalPCA(n_components=LATENT_DIM, batch_size=1)
)
#model7=compose.Pipeline(
#    preprocessing.MinMaxScaler(),
#    SklearnAnomalyDetector(IncrementalPCA(n_components=10))
#)
#rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
#for x, y in data_stream:
#    print(x.values())
#data_stream = stream.shuffle(river.datasets.CreditCard().take(20000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
#data_stream = list(data_stream)

data1 = pd.DataFrame(data=data_stream)
#data2 = pd.DataFrame.from_dict(data=data1,orient='index')
#test=data1[0].to_dict()
#df_test=pd.DataFrame.from_dict(test, orient='index')
#df_test
#pd.DataFrame.from_dict({(i,j): data1[i][j] 
#                           for i in data1.keys() 
#                           for j in data1[i].keys()},
#                       orient='index')
#data_stream = stream.shuffle(river.datasets.CreditCard().take(20000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
data1 = pd.DataFrame(data=data_stream)
df_test= pd.DataFrame.from_dict(data1)


data_stream1 = stream.shuffle(df_test.itertuples(index=False),N_SAMPLES, seed=SEED)
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
#df=pd.read_csv(r'C:\Users\Manuel\river_data\CreditCard\creditcard.csv')
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(5000), N_SAMPLES, seed=40)
MSE_score_arr = []
values = []
for x, y in data_stream:
    #print(pd.DataFrame.from_dict(x))
    ipca = ipca.partial_fit(pd.DataFrame.from_dict([x]))
    x_trans = ipca.transform(pd.DataFrame.from_dict([x]))
    inverse_trans= ipca.inverse_transform(x_trans)
    MSE_score = ((np.array(pd.DataFrame.from_dict([x]))-np.array([inverse_trans]))**2).sum()
    MSE_score_arr.append(MSE_score)
    values.append(y)
#    model7.learn_one(x)
#    y_pred= model7.score_one(x)
#    rocauc.update(y,y_pred)
rocauc
values_df = pd.Series(data=values)
loss = pd.Series(data=MSE_score_arr)
loss = (loss-np.min(loss))/(np.max(loss)-np.min(loss))
loss.sum()
rocauc = metrics.ROCAUC()
for yt, yp in zip(values, loss):
    #print(yt, yp)
    rocauc = rocauc.update(yt, yp)
rocauc
#evaluation.append(rocauc)

C:\Users\Manuel\Anaconda3\envs\rwch\lib\site-packages\sklearn\decomposition\_incremental_pca.py:337: RuntimeWarning: invalid value encountered in true_divide
  explained_variance = S ** 2 / (n_total_samples - 1)
C:\Users\Manuel\Anaconda3\envs\rwch\lib\site-packages\sklearn\decomposition\_incremental_pca.py:338: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
C:\Users\Manuel\Anaconda3\envs\rwch\lib\site-packages\sklearn\decomposition\_incremental_pca.py:348: RuntimeWarning: Mean of empty slice.
  self.noise_variance_ = explained_variance[self.n_components_ :].mean()
C:\Users\Manuel\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ROCAUC: 0.499556

### SGDRegressor (SVM-like with Kernel)

In [78]:
from sklearn.linear_model import SGDClassifier
from river import evaluate
x_arr=[]
y_arr=[]
y_pred_arr=[]

data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)


model6=compose.Pipeline(
    preprocessing.StandardScaler(),
    fx.RBFSampler(),
    compat.convert_sklearn_to_river(SGDClassifier(),classes=[False, True]),
    )

rocauc= river.metrics.ROCAUC(n_thresholds=100)

for x, y in data_stream:
    y_pred= model6.predict_one(x)
    model6.learn_one(x,y)       
    rocauc.update(y,y_pred)
    x_arr.append([x])
    y_arr.append(y)
    y_pred_arr.append([y,y_pred])
rocauc

ROCAUC: 0.960653

In [79]:
#test=pd.DataFrame(y_pred_arr,columns=['y_true','y_pred'])
# test.sort_values(by=['y_true'], ascending=False).head(50)
#test.sort_values(by=['y_pred'], ascending=False).head(50)

### HoeffdingAdaptiveTreeClassifier

In [96]:
from river.tree import HoeffdingAdaptiveTreeClassifier

In [99]:
model8 = compose.Pipeline(
    preprocessing.StandardScaler(),
    HoeffdingAdaptiveTreeClassifier()
)

In [100]:
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
i=0
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    #print(y)
    if i==0:
        model8.learn_one(x,y)
        i=i+1
    y_pred=model8.predict_one(x)
    #print(y_pred)
    model8.learn_one(x,y)
    rocauc.update(y,y_pred)
rocauc
evaluation=evaluation.append({
    'model':'HalfSpaceTrees_min_max_scaler',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)

In [101]:
rocauc

ROCAUC: 0.944648

### Logistic Regression

In [105]:
from river import optim as op

model9 = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LogisticRegression(optimizer=op.SGD(0.1))
)

In [106]:
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=42)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
for x, y in data_stream:
    y_pred=model9.predict_one(x)
    model9.learn_one(x,y)
    rocauc.update(y,y_pred)
#evaluation.append(rocauc)
rocauc
evaluation=evaluation.append({
    'model':'HalfSpaceTrees_min_max_scaler',
    'anom_percentage':anom_percentage,
    'num_samples':num_samples,
    'Loss':LOSS,
    'Optimizer':OPTIMIZER,
    'Batch_size':BATCH_SIZE,
    'Learning_rate':LEARNING_RATE,
    'Latent_dim': LATENT_DIM,
    'ROC':rocauc,
    #'Precision_Recall_Curve':45,
    #'FP':5,
    #'TP': 4,
    #'FN':8,
    #'TN':10

},ignore_index=True)

In [107]:
rocauc

ROCAUC: 0.971211

# Random

## PCA

In [ ]:
'''ipca = IncrementalPCA(n_components=LATENT_DIM)
model6=compose.Pipeline(
    preprocessing.MinMaxScaler(),
    ipca #IncrementalPCA(n_components=LATENT_DIM, batch_size=1)
)
#model7=compose.Pipeline(
#    preprocessing.MinMaxScaler(),
#    SklearnAnomalyDetector(IncrementalPCA(n_components=10))
#)
#rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
#for x, y in data_stream:
#    print(x.values())
#data_stream = stream.shuffle(river.datasets.CreditCard().take(20000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
#data_stream = list(data_stream)

data1 = pd.DataFrame(data=data_stream)
#data2 = pd.DataFrame.from_dict(data=data1,orient='index')
#test=data1[0].to_dict()
#df_test=pd.DataFrame.from_dict(test, orient='index')
#df_test
#pd.DataFrame.from_dict({(i,j): data1[i][j] 
#                           for i in data1.keys() 
#                           for j in data1[i].keys()},
#                       orient='index')
#data_stream = stream.shuffle(river.datasets.CreditCard().take(20000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
data1 = pd.DataFrame(data=data_stream)
df_test= pd.DataFrame.from_dict(data1)


data_stream1 = stream.shuffle(df_test.itertuples(index=False),N_SAMPLES, seed=SEED)
#data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), N_SAMPLES, seed=SEED)
data_stream = stream.shuffle(final_set.itertuples(index=False),N_SAMPLES, seed=SEED)
#df=pd.read_csv(r'C:\Users\Manuel\river_data\CreditCard\creditcard.csv')
rocauc= river.metrics.ROCAUC()
#data_stream = stream.shuffle(river.datasets.CreditCard().take(5000), N_SAMPLES, seed=40)
MSE_score_arr = []
values = []
for x, y in data_stream:
    #print(pd.DataFrame.from_dict(x))
    ipca = ipca.partial_fit(pd.DataFrame.from_dict([x]))
    x_trans = ipca.transform(pd.DataFrame.from_dict([x]))
    inverse_trans= ipca.inverse_transform(x_trans)
    MSE_score = ((np.array(pd.DataFrame.from_dict([x]))-np.array([inverse_trans]))**2).sum()
    MSE_score_arr.append(MSE_score)
    values.append(y)
#    model7.learn_one(x)
#    y_pred= model7.score_one(x)
#    rocauc.update(y,y_pred)
rocauc
values_df = pd.Series(data=values)
loss = pd.Series(data=MSE_score_arr)
loss = (loss-np.min(loss))/(np.max(loss)-np.min(loss))
loss.sum()
rocauc = metrics.ROCAUC()
for yt, yp in zip(values, loss):
    #print(yt, yp)
    rocauc = rocauc.update(yt, yp)
rocauc
#evaluation.append(rocauc)
'''

In [ ]:
'''dataset = datasets.synth.RandomRBF(seed_model=7, seed_sample=SEED,n_classes=2,n_features=200).take(N_SAMPLES)
data_stream = stream.shuffle(dataset,buffer_size=N_SAMPLES)

y_count=0
y_0_count=0

def build_fn(n_features):
    net = nn.Sequential(
        nn.Linear(n_features, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 1),
        nn.Softmax()
    )
    return net

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    PyTorch2RiverClassifier(
                build_fn=build_fn,
                loss_fn=nn.BCELoss,
                optimizer_fn=optim.Adam,
                batch_size=1,
                learning_rate=1e-3,
    )
)

for x, y in data_stream:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x)
    rocauc.update(y, y_pred)
    #print(y)
    if y==1:
        y_count+=1
    else:
        y_0_count+=1
rocauc'''

C:\Users\Manuel\Anaconda3\envs\rwch\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ROCAUC: 0.499488

In [ ]:
#y_count

539

In [ ]:
#y_0_count

461